<a href="https://colab.research.google.com/github/bennng/Spark/blob/master/Linear_Regression_Code_Along.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression Code Along

This notebook is the reference for the video lecture on the Linear Regression Code Along. Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

First thing to do is start a Spark Session

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [0]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [0]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank Tunnel
...|          Violet| 34.49726772511229|12.655651149166752| 39.57766801952616|   4.082620632952961|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen|31.926272026360156|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512| 37.11059744212085|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [0]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank Tunnel\nWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.655651149166752, Time on Website=39.57766801952616, Length of Membership=4.082620632952961, Yearly Amount Spent=587.9510539684005)

In [0]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank Tunnel
Wrightmouth, MI 82180-9605
Violet
34.49726772511229
12.655651149166752
39.57766801952616
4.082620632952961
587.9510539684005


## Setting Up DataFrame for Machine Learning 

In [0]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [0]:
output = assembler.transform(data)

In [0]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [0]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank Tunnel
...|          Violet| 34.49726772511229|12.655651149166752| 39.57766801952616|   4.082620632952961|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen|31.926272026360156|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512| 3

In [0]:
final_data = output.select("features",'Yearly Amount Spent')

In [0]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                374|
|   mean| 494.77136952135584|
| stddev|  79.32347347332397|
|    min| 256.67058229005585|
|    max|  765.5184619388372|
+-------+-------------------+



In [0]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                126|
|   mean|  512.7978327643508|
| stddev|  78.05150254141621|
|    min|  275.9184206503857|
|    max|  689.2356997616951|
+-------+-------------------+



In [0]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [0]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

In [0]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.9313842258,38.5223901033,0.061323831352,61.8542031461] Intercept: -1042.5967496966575


In [0]:
test_results = lrModel.evaluate(test_data)

In [0]:
# Interesting results....
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  0.0211314098588673|
|   5.133420657904367|
| -22.148174773562573|
|-0.28200824770379995|
|  2.6981651070976795|
|  0.7429079096369264|
|  -3.455359493022627|
| -2.1941889835831034|
|   7.147573069494342|
|   1.206688448485238|
| -12.184920705516333|
|  0.9395486603143581|
|   7.893631818058566|
| -1.6946216546640471|
|  -9.987986847356694|
|  -6.281630483941342|
|  -9.000712269275198|
|  -18.16281648837412|
|  -9.055807893457029|
|    3.71115734739368|
+--------------------+
only showing top 20 rows



In [0]:
unlabeled_data = test_data.select('features')

In [0]:
predictions = lrModel.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...|442.04328234820673|
|[31.0472221394875...|  387.363978531117|
|[31.1239743499119...|509.09522861332835|
|[31.2606468698795...|421.60863950465523|
|[31.3091926408918...|430.02255273283595|
|[31.3895854806644...|  409.326703150346|
|[31.5171218025062...|279.37378014340834|
|[31.5761319713222...| 543.4207729729114|
|[31.6548096756927...| 468.1158506580541|
|[31.7366356860502...| 495.7267578070466|
|[31.8093003166791...| 548.9568200683575|
|[31.8293464559211...|384.21278932766063|
|[31.8512531286083...|465.09861484873977|
|[31.8627411090001...| 557.9927628287107|
|[31.8648325480987...| 449.8792673241703|
|[31.8745516945853...|398.56687473020884|
|[31.8854062999117...| 399.1039852417507|
|[31.9563005605233...| 565.2887482355729|
|[32.0085045178551...| 452.2530289222125|
|[32.0123007682454...|489.23389571856455|
+--------------------+------------

In [0]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 10.744845638086671
MSE: 115.45170778631018


Excellent results! Let's see how you handle some more realistically modeled data in the Consulting Project!